In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow.contrib.slim as slim


mnist = input_data.read_data_sets('/tmp/mnist/data',one_hot=True)

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 28*28], name='input')
y_actual = tf.placeholder(tf.float32, shape=[None, 10], name='label')
is_training = tf.placeholder(tf.bool, name='is_training')

def convnet(inputs, is_training, scope='Convnet'):
    with tf.variable_scope(scope, 'Convnet'):
        inputs= tf.reshape(inputs, [-1,28,28,1])
        # First Group: Convolution + Pooling 28x28x1 => 28x28x20 => 14x14x20
        net = slim.conv2d(inputs, 20, [5, 5], padding='SAME', scope='layer1-conv')
        net = slim.max_pool2d(net, 2, stride=2, scope='layer2-max-pool')

        # Second Group: Convolution + Pooling 14x14x20 => 10x10x40 => 5x5x40
        net = slim.conv2d(net, 40, [5, 5], padding='VALID', scope='layer3-conv')
        net = slim.max_pool2d(net, 2, stride=2, scope='layer4-max-pool')

        # Reshape: 5x5x40 => 1000x1
        net = slim.flatten(net)

        # Fully Connected Layer: 1000x1 => 1000x1
        net = slim.fully_connected(net, 1000, scope='layer5')
        net = slim.dropout(net,keep_prob=0.5, is_training=is_training, scope='layer5-dropout')

        # Second Fully Connected: 1000x1 => 1000x1
        net = slim.fully_connected(net, 1000, scope='layer6')
        net = slim.dropout(net,keep_prob=0.5, is_training=is_training, scope='layer6-dropout')

        # Output Layer: 1000x1 => 10x1
        net = slim.fully_connected(net, 10, scope='output')
        net = slim.dropout(net,keep_prob=0.5, is_training=is_training, scope='output-dropout')

        return net

logits = convnet(x, is_training, scope='Covnet')

with tf.name_scope('Loss'):
	cross_entropy =  slim.losses.softmax_cross_entropy(logits, y_actual)
with tf.name_scope('Prediction'):
	correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_actual, 1))
	accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
with tf.name_scope('Optimizer'):
    train_step = tf.train.MomentumOptimizer(0.03,0.5).minimize(cross_entropy)

sess.run(tf.global_variables_initializer())

val_data = {
    x: mnist.validation.images,
    y_actual: mnist.validation.labels,
    is_training: False
}

for i in range(5000):
    images, labels = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: images, y_actual: labels, is_training: True})
    
    if i % 100 == 0:
        acc = sess.run( accuracy, feed_dict= val_data)
        print("Step: %5d,CV Accuracy = %5.2f%%" % (i, acc * 100))

test_data = {
    x: mnist.test.images,
    y_actual: mnist.test.labels,
    is_training: False
}

acc = sess.run(accuracy, feed_dict= test_data)

print("Test Accuracy is %5.2f%%" % (100 * acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official

Step:  1600,CV Accuracy = 97.58%
Step:  1700,CV Accuracy = 97.84%
Step:  1800,CV Accuracy = 98.04%
Step:  1900,CV Accuracy = 97.96%
Step:  2000,CV Accuracy = 98.12%
Step:  2100,CV Accuracy = 98.10%
Step:  2200,CV Accuracy = 98.34%
Step:  2300,CV Accuracy = 98.04%
Step:  2400,CV Accuracy = 98.40%
Step:  2500,CV Accuracy = 98.42%
Step:  2600,CV Accuracy = 98.32%
Step:  2700,CV Accuracy = 98.54%
Step:  2800,CV Accuracy = 98.58%
Step:  2900,CV Accuracy = 98.56%
Step:  3000,CV Accuracy = 98.60%
Step:  3100,CV Accuracy = 98.66%
Step:  3200,CV Accuracy = 98.48%
Step:  3300,CV Accuracy = 98.42%
Step:  3400,CV Accuracy = 98.60%
Step:  3500,CV Accuracy = 98.74%
Step:  3600,CV Accuracy = 98.74%
Step:  3700,CV Accuracy = 98.74%
Step:  3800,CV Accuracy = 98.74%
Step:  3900,CV Accuracy = 98.74%
Step:  4000,CV Accuracy = 98.80%
Step:  4100,CV Accuracy = 98.68%
Step:  4200,CV Accuracy = 98.82%
Step:  4300,CV Accuracy = 98.80%
Step:  4400,CV Accuracy = 98.84%
Step:  4500,CV Accuracy = 98.76%
Step:  460